<a href="https://colab.research.google.com/github/coldsober-irene/NLP-fellowship/blob/main/scrap_igihe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup as bs
from google.colab import drive
import pandas as pd
import numpy as np
import re
from time import strftime

create the sequencial (daily) links from january to december 2022

In [ ]:
validLinks = []
current_month = int(strftime("%m")) + 1
for month in range(1, current_month):
  for day in range(1, 32):
    l = "http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}".format(month, day)
    response = requests.get(l).json()
    if  response['archived_snapshots']:
      validLinks.append(response['archived_snapshots']['closest']["url"])

check if they are some duplicated links and get rid of them

In [ ]:
arr = np.array(validLinks)
series = pd.Series(arr)
validLinks = series.unique()

In [ ]:
article_links = {}

get every article link and its date of creation

In [ ]:
def process_links(link):
  page = requests.get(link)
  soup = bs(page.text, "html.parser")
  
  b = soup.body
  get_titles = [link+re.sub("amakuru/","", a['href']) for span in b.find_all("span", class_ = "homenews-title") for a in span.find_all("a")]
  
  # get only links whose date is in current year
  pattern = re.compile(r"\d{8,}?")
  found_date = pattern.search(link).group()
  if found_date[3] != str(2):
    pass
  else:
    file_title = f"igihe-{found_date[0:4]}-{found_date[4:6]}-{found_date[6:8]}"
    try:
      for title in get_titles:
        article_links[file_title].append(title)
    except KeyError:
      article_links[file_title] = []
      for title in get_titles:
        article_links[file_title].append(title)

In [ ]:
for link in validLinks:
  process_links(link)

function to write article's paragraphs to a file corresponding to the article's creation date

In [ ]:
def writeToFile(paragraphs, file):
  with open(file, "a+") as f:
    for paragraph in paragraphs:
      f.write(paragraph)

a function to grab the article url and return the article's text i.e. paragraphs and call the function to write those paragraphs to their corresponding file

In [ ]:
def get_page():
  count = 1
  all_file = list(article_links.keys())
  for key in list(article_links.keys()):
    for link in set(article_links[key]):
      p = requests.get(link).content
      soup = bs(p, "html.parser")
      spans = soup.find_all("div", class_= "fulltext margintop10")
      paragraphs = [p.get_text() for span in spans for p in span.find_all("p")]
      writeToFile(paragraphs = paragraphs, file = f"/content/drive/MyDrive/NLP fellowship/week1 tasks/{key}.txt")
    # 
    print(f"{count}file(s) created \tremains : {all_file - count} files")
    count += 1

finish the program here

In [ ]:
# FINISH THE PROGRAM
get_page()